Derek Yadgaroff

Assignment 3, Exercise 4

21VT-2DV516

In [20]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import mode

In [3]:
# Load data
data = np.loadtxt('./data/bm.csv',delimiter=',')
X = data[:, 0:-1]
y = data[:, -1]

In [4]:
# Set seed
rng = np.random.default_rng(3)
permutation = rng.permutation(X.shape[0])

In [5]:
# Randomize Data
X_randomized = X[permutation]
y_randomized = y[permutation]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_randomized,
                                                    y_randomized,
                                                    train_size = .5, shuffle=False)

In [6]:
# Set subsets
n = 5000
num_trees= 100

# Create subset indices here so we can reproduce results easily
q_list = []
for i in range(num_trees):
    q = rng.choice(n, n, True)
    q_list.append(q)

In [7]:
# Create list of data to be used by each tree in the ensmble

X_train_list = []
y_train_list = []

for q in q_list:
    X_train_i = X_train[q]
    y_train_i = y_train[q]
    
    X_train_list.append(X_train_i)
    y_train_list.append(y_train_i)

In [32]:
# Create list of individual tree predictions which will make up the ensemble
z_list = []

# Tally total score of individual trees
ind_total_score = 0

# Create, fit, predict for each individual tree , tally score and accuracy
for X_train_tree, y_train_tree in zip(X_train_list,y_train_list):
    clf = DecisionTreeClassifier()
    clf.fit(X_train_tree, y_train_tree)
    z = clf.predict(X_test)
    score = np.sum(z==y_test)
    ind_total_score += score
    acc = score/n
    z_list.append(z)
ind_total_acc = ind_total_score/(n*num_trees)

In [33]:
# Convert list of trees's predictions to a matrix (the ensemble)
ensemble = np.array(z_list).T
z_ensemble, count = mode(ensemble,axis=1)
z_corr = np.sum(z_ensemble.reshape(-1,) == y_test)
z_acc = z_corr/n


### Question a

In [34]:
print(z_acc)

0.9778


### Question b

In [35]:

print(ind_total_acc)

0.963518


### Question c

### Question d

In [ ]:
# Ex A.2
# rbf = gaussian,
clf = SVC(kernel="rbf", gamma=.5, C=20)
clf.fit(X_train, y_train)
print("Ex A_2, accuracy:",clf.score(X_train, y_train))

# Ex A.3

# Separate vectors
X1 = X_train[clf.support_, 0]
X2 = X_train[clf.support_, 1]
y_pred = y_train[clf.support_]

# Meshgrid
x_min, x_max = X1.min() - 0.1, X1.max() + 0.1
y_min, y_max = X2.min() - 0.1, X2.max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, .01),
                     np.arange(y_min, y_max, .01))

# plot boundary with support vector    
fig = plt.figure()
fig.suptitle("Ex A, Decision boundary with support vector")
ax = fig.add_subplot(1, 1, 1)
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Map of predictions
Z = Z.reshape(xx.shape)

# Show the boundary
ax.contour(xx, yy, Z, colors='r')

# plot support vectors
ax.scatter(X1, X2, s=1,c=y_pred)

# plot boundary with data    
fig = plt.figure()
fig.suptitle("Ex A, Decision boundary with data")
ax = fig.add_subplot(1, 1, 1)
ax.contour(xx, yy, Z,colors='r')
ax.scatter(X_train[:,0], X_train[:,1], s=.5,c=y_train)
plt.show()
